# Mochilas Multiplas
![text](mochilas_multiplas.PNG)

In [73]:
import gurobipy as gp

#parametros do problema
qtd_itens=63
qtd_mochilas=3

#vetores com pesos, valores e capacidades
vet_pesos = [145,159,148,128,80,92,141,178,93,146,64,129,83,84,127,168,75,80,153,133,110,79,105,87,87,53,158,146,51,107,116,165,167,134,169,62,57,127,54,199,115,108,138,81,75,169,148,191,82,123,189,189,146,55,140,88,154,157,106,53,192,180,177,]
vet_valores = [326,417,333,345,202,209,363,404,232,350,172,309,213,201,311,424,174,204,397,302,253,215,270,230,221,125,430,397,132,272,281,400,375,314,447,150,128,339,139,542,290,251,317,212,168,452,399,453,223,319,505,448,365,127,318,200,400,365,262,137,441,423,469]
vet_capacidades = [1500,2000,1700]

#Rotulo dos items
itens = list()
for i in range(qtd_itens):
    itens.append(f"Item_{i+1}")

#Rotulo das mochilas
mochilas = list()
for i in range(qtd_mochilas):
    mochilas.append(f"Mochila_{i+1}")

#Dicionario de capacidades
capacidades = dict()
for idx, cap in enumerate(vet_capacidades):
    rotulo = mochilas[idx]
    capacidades[rotulo] = cap

#Dict pesos e vlores
pesos = dict()
valores = dict()
for i in range(qtd_itens):
    rotulo = itens[i]
    pesos[rotulo] = vet_pesos[i]
    valores[rotulo] = vet_valores[i]

#criando o modelo
m = gp.Model()
m.setParam(gp.GRB.Param.OutputFlag,0)
#Insere as variaveis de decisao
#Nesse problema agora, com dois indices i,j
x = m.addVars(itens,mochilas, vtype = gp.GRB.BINARY) #pra todas as combinações de items e mochilas ele vai criar uma variavel

#Função objetivo

m.setObjective(
    gp.quicksum(x[i,j] * valores[i] for i in itens for j in mochilas),
    sense=gp.GRB.MAXIMIZE
)

#Restrição 1
c1 = m.addConstrs(
    gp.quicksum( x[i,j] for j in mochilas) <= 1  for i in itens
)

#Restrição 1
c2 = m.addConstrs(
    gp.quicksum( pesos[i]*x[i,j] for i in itens) <= capacidades[j]  for j in mochilas
)

#Executando o modelo
m.optimize()

#Mostrando resultados
#Valor Objetivo
print("Valor Total", m.objVal)

#Itens das mochilas
for j in mochilas:
    print(j)
    for i in itens:
        if round(x[i,j].X) == 1:
            print(i, " ", end="")

    print("\n")
for j in mochilas:
    print(f"Capacidade utilizada: {capacidades[j] - c2[j].Slack}/{capacidades[j]}")

Valor Total 13364.0
Mochila_1
Item_4  Item_10  Item_12  Item_13  Item_14  Item_15  Item_18  Item_19  Item_23  Item_24  Item_35  Item_38  Item_49  

Mochila_2
Item_5  Item_16  Item_22  Item_31  Item_32  Item_40  Item_41  Item_47  Item_50  Item_51  Item_57  Item_59  Item_62  Item_63  

Mochila_3
Item_2  Item_7  Item_9  Item_11  Item_25  Item_27  Item_28  Item_29  Item_30  Item_39  Item_44  Item_46  Item_48  Item_53  Item_60  

Capacidade utilizada: 1500.0/1500
Capacidade utilizada: 1999.0/2000
Capacidade utilizada: 1700.0/1700
